# Inserimento dei dati nel database tramite foglio excell
### Matter Of Design

In [ ]:
!pip install pandas openpyxl mysql-connector-python

In [1]:
# Librerie necessarie
import pandas as pd
import mysql.connector


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\emmal\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\emmal\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\emmal\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", 

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\emmal\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\emmal\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\emmal\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", 

AttributeError: _ARRAY_API not found

: 

## Caricamento dati

In [ ]:
# Funzione per connettersi al database
def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host='matterofdesign.cboqqic8cvp0.eu-west-2.rds.amazonaws.com',
            user='Massaro',
            password='4PGZUfbq^$pbngQ',
            database='MatterOfDesign'
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return None

# Funzione aggiornata per caricare i dati
def carica_dati(file_path, foglio, tipologia_id):
    connection = connect_to_db()
    if connection is None:
        print("Errore nella connessione al database.")
        return

    cursor = connection.cursor()

    try:
        # Leggi il foglio Excel
        df = pd.read_excel(file_path, sheet_name=foglio)
        df.dropna(how='all', inplace=True)  # Elimina righe completamente vuote

        # Verifica la presenza della colonna codice_articolo
        if "codice_articolo" not in df.columns:
            print(f"Il foglio '{foglio}' non contiene la colonna 'codice_articolo'.")
            return

        # Mantieni solo righe con codice_articolo non vuoto
        df = df[df["codice_articolo"].notnull()]

        if df.empty:
            print(f"Il foglio '{foglio}' non contiene elementi validi. Salto il caricamento.")
            return

        # Aggiungi Tipologia_ID
        df["tipologia_id"] = tipologia_id

        # Lista delle colonne richieste
        colonne_richieste = [
            "codice_articolo", "brand", "finitura", "posa", "collezione",
            "dimensione", "prezzo_unitario", "prezzo_mq", "colore", "spessore",
            "documento", "immagine", "nota", "peso", "categoria"
        ]

        # Aggiungi colonne mancanti con valore None
        for colonna in colonne_richieste:
            if colonna not in df.columns:
                df[colonna] = None

        # Converti NaN in None (importante per MySQL)
        df = df.where(pd.notnull(df), None)

        # Verifica che tutte le colonne siano in formato stringa o compatibile
        df = df.astype(object).replace({pd.NA: None})

        # Query di controllo per evitare duplicati
        check_query = "SELECT COUNT(*) FROM Prodotto WHERE codice_articolo = %s AND tipologia_id = %s"

        # Query di inserimento
        insert_query = """
        INSERT INTO Prodotto (codice_articolo, brand, finitura, posa, collezione, dimensione,
        prezzo_unitario, prezzo_mq, colore, spessore, tipologia_id, documento, immagine, nota,
        peso, categoria)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Inserisci i dati
        for _, row in df.iterrows():
            codice_articolo = row["codice_articolo"]

            # Verifica se il prodotto esiste già
            cursor.execute(check_query, (codice_articolo, tipologia_id))
            result = cursor.fetchone()

            if result[0] == 0:  # Se il prodotto non esiste già
                data = (
                    row["codice_articolo"], row["brand"], row["finitura"], row["posa"],
                    row["collezione"], row["dimensione"], row["prezzo_unitario"],
                    row["prezzo_mq"], row["colore"], row["spessore"], row["tipologia_id"],
                    row["documento"], row["immagine"], row["nota"], row["peso"], row["categoria"]
                )

                # Inserimento nel database
                cursor.execute(insert_query, data)
                print(f"Inserito prodotto con codice_articolo: {codice_articolo}")

        # Conferma modifiche
        connection.commit()
        print(f"Dati caricati per la tipologia '{foglio}' (ID: {tipologia_id}) con successo!")

    except Exception as e:
        print(f"Errore durante l'elaborazione del foglio '{foglio}': {e}")

    finally:
        cursor.close()
        connection.close()



# Percorso del file Excel
file_path = r"C:/Users/emmal/Desktop/STAGE/PROGETTO/HowToDo/Listino.xlsx"

# Dizionario foglio-tipologia
foglio_to_tipologia_id = {
    "PAVIMENTO_LEGNO": 1,
    "PAVIMENTO": 2,
    "RIVESTIMENTI": 3,
    "SANITARI": 4,
    "RUBINETTERIA": 5,
    "VASCHE_BAGNO": 6,
    "PIATTI_DOCCIA": 7,
    "PORTE_INTERNE": 8,
    "TERMOARREDI": 9,
    "WELLNESS": 10,
    "LAVABI": 11,
}

# Caricamento fogli
for foglio, tipologia_id in foglio_to_tipologia_id.items():
    print(f"Caricamento del foglio: {foglio}, ID Tipologia: {tipologia_id}")
    carica_dati(file_path, foglio, tipologia_id)


Caricamento del foglio: PAVIMENTO_LEGNO, ID Tipologia: 1


## Verifica dati

In [ ]:
def fetch_data():
    connection = connect_to_db()
    if connection is None:
        print("Errore nella connessione al database.")
        return

    cursor = connection.cursor()

    try:
        cursor.execute("SELECT * FROM Prodotto")
        results = cursor.fetchall()

        if not results:
            print("La tabella 'Prodotto' è vuota o non ci sono dati.")
            return

        # Ottieni i nomi delle colonne
        column_names = [desc[0] for desc in cursor.description]
        print("Dati nella tabella 'Prodotto':")
        print("-" * 160)  # Larghezza adattata per tutte le colonne
        print(" | ".join(f"{col:<15}" for col in column_names))
        print("-" * 160)

        for row in results:
            # Sostituisci valori None con "N/A" per evitare errori di formattazione
            formatted_row = [value if value is not None else "N/A" for value in row]
            print(" | ".join(f"{str(col):<15}" for col in formatted_row))

    except Exception as e:
        print(f"Errore durante il recupero dei dati: {e}")
    finally:
        cursor.close()
        connection.close()


# Esegui la funzione per vedere i dati
fetch_data()
